In [31]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam

import numpy as np
import random
from collections import deque

class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = load_model("/home/tushar/Desktop/Python/Python3/q-trader/models/" + model_name) if is_eval else self._model()

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))

        return model

    def act(self, state):
        if not self.is_eval and np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])

    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])

        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay 

In [32]:
import numpy as np
import math

# prints formatted price
def formatPrice(n):
    return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

# returns the vector containing stock data from a fixed file
def getStockDataVec(key):
    vec = []
    #lines = open("data/" + key + ".csv", "r").read().splitlines()
    lines = open(key, "r").read().splitlines()

    for line in lines[1:]:
        vec.append(float(line.split(",")[4]))

    return vec

# returns the sigmoid
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# returns an an n-day state representation ending at time t
def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
    return np.array([res])

In [ ]:
import sys

#if len(sys.argv) != 4:
#	print "Usage: python train.py [stock] [window] [episodes]"
#	exit()

#stock_name, window_size, episode_count = sys.argv[1], int(sys.argv[2]), int(sys.argv[3])

window_size = 30
stock_name = "/home/tushar/Desktop/Python/Python3/q-trader/data/^GSPC.csv"
episode_count = 200

agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
    print ("Episode: {} / {} ".format(str(e),str(episode_count)))
    state = getState(data, 0, window_size + 1)
    total_profit = 0
    agent.inventory = []

    for t in range(l):
        action = agent.act(state)
        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0

        if action == 1: # buy
            agent.inventory.append(data[t])
            print ("Buy: {}".format(formatPrice(data[t])))

        elif action == 2 and len(agent.inventory) > 0: # sell
            bought_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print ("Sell: {} | Profit: {} ".format(formatPrice(data[t]),formatPrice(data[t] - bought_price))) 

        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            print ("--------------------------------")
            print ("Total Profit: {}".format(formatPrice(total_profit)))  
            print ("--------------------------------")

        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)

    if e % 10 == 0:
        agent.model.save("/home/tushar/Desktop/Python/Python3/q-trader/models/model_ep" + str(e))

Episode: 0 / 200 
Buy: $1333.34
Sell: $1295.86 | Profit: -$37.48 
Buy: $1326.82
Sell: $1326.65 | Profit: -$0.17 
Buy: $1342.54
Sell: $1342.90 | Profit: $0.36 
Buy: $1354.95
Sell: $1373.73 | Profit: $18.78 
Buy: $1373.47
Sell: $1349.47 | Profit: -$24.00 
Buy: $1354.31
Buy: $1352.26
Buy: $1340.89
Sell: $1332.53 | Profit: -$21.78 
Buy: $1314.76
Sell: $1330.31 | Profit: -$21.95 
Sell: $1318.80 | Profit: -$22.09 
Sell: $1315.92 | Profit: $1.16 
Buy: $1326.61
Sell: $1301.53 | Profit: -$25.08 
Buy: $1278.94
Sell: $1255.27 | Profit: -$23.67 
Buy: $1245.86
Sell: $1267.65 | Profit: $21.79 
Buy: $1234.18
Buy: $1241.41
Sell: $1261.89 | Profit: $27.71 
Sell: $1264.74 | Profit: $23.33 
Buy: $1233.42
Buy: $1166.71
Buy: $1173.56
Buy: $1150.53
Buy: $1142.62
Buy: $1117.58
Buy: $1139.83
Sell: $1152.69 | Profit: -$80.73 
Buy: $1153.29
Sell: $1160.33 | Profit: -$6.38 
Buy: $1145.87
Sell: $1103.25 | Profit: -$70.31 
Sell: $1151.44 | Profit: $0.91 
Sell: $1183.50 | Profit: $40.88 
Buy: $1191.81
Sell: $1238.1

Sell: $1195.90 | Profit: -$35.12 
Sell: $1187.33 | Profit: -$22.87 
Sell: $1184.87 | Profit: -$30.79 
Sell: $1177.68 | Profit: -$39.21 
Sell: $1176.84 | Profit: -$51.97 
Sell: $1186.57 | Profit: -$27.90 
Buy: $1179.59
Buy: $1199.38
Buy: $1196.54
Buy: $1191.38
Buy: $1178.90
Sell: $1198.41 | Profit: $18.82 
Sell: $1207.01 | Profit: $7.63 
Sell: $1202.76 | Profit: $6.22 
Sell: $1214.76 | Profit: $23.38 
Sell: $1219.94 | Profit: $41.04 
Buy: $1259.37
Buy: $1260.43
Buy: $1267.43
Buy: $1272.74
Buy: $1270.94
Buy: $1267.32
Buy: $1259.92
Buy: $1259.62
Sell: $1262.79 | Profit: $3.42 
Sell: $1268.12 | Profit: $7.69 
Sell: $1268.66 | Profit: $1.23 
Sell: $1256.54 | Profit: -$16.20 
Sell: $1258.17 | Profit: -$12.77 
Sell: $1254.42 | Profit: -$12.90 
Sell: $1248.29 | Profit: -$11.63 
Buy: $1268.80
Buy: $1273.46
Buy: $1273.48
Buy: $1285.45
Buy: $1290.15
Buy: $1289.69
Sell: $1294.18 | Profit: $34.56 
Sell: $1286.06 | Profit: $17.26 
Sell: $1287.61 | Profit: $14.15 
Sell: $1282.93 | Profit: $9.45 
Sell

Buy: $1328.32
Buy: $1375.94
Sell: $1284.91 | Profit: -$155.79 
Sell: $1262.90 | Profit: -$144.32 
Sell: $1273.70 | Profit: -$154.53 
Sell: $1244.69 | Profit: -$224.33 
Sell: $1253.39 | Profit: -$216.33 
Sell: $1239.49 | Profit: -$241.65 
Sell: $1228.30 | Profit: -$244.12 
Sell: $1214.91 | Profit: -$247.88 
Sell: $1245.36 | Profit: -$239.65 
Sell: $1260.32 | Profit: -$247.02 
Sell: $1260.68 | Profit: -$243.98 
Sell: $1260.00 | Profit: -$255.96 
Sell: $1277.00 | Profit: -$200.65 
Sell: $1282.19 | Profit: -$206.22 
Sell: $1252.54 | Profit: -$215.41 
Sell: $1257.76 | Profit: -$188.14 
Sell: $1234.37 | Profit: -$220.61 
Sell: $1263.20 | Profit: -$115.35 
Sell: $1284.26 | Profit: -$111.16 
Sell: $1267.38 | Profit: -$113.44 
Sell: $1260.31 | Profit: -$76.33 
Sell: $1249.01 | Profit: -$77.44 
Sell: $1284.88 | Profit: -$52.03 
Sell: $1289.19 | Profit: -$42.10 
Sell: $1266.07 | Profit: -$73.06 
Sell: $1296.32 | Profit: -$52.46 
Sell: $1305.32 | Profit: -$37.21 
Sell: $1289.59 | Profit: -$63.52 


Buy: $936.31
Buy: $934.53
Buy: $920.75
Sell: $917.58 | Profit: $38.38 
Sell: $906.55 | Profit: $22.16 
Sell: $912.23 | Profit: $12.51 
Sell: $892.00 | Profit: $1.84 
Sell: $904.45 | Profit: $8.31 
Sell: $904.96 | Profit: $22.46 
Sell: $901.58 | Profit: $3.93 
Sell: $889.48 | Profit: -$0.75 
Sell: $910.40 | Profit: $28.25 
Sell: $902.99 | Profit: $12.28 
Sell: $891.12 | Profit: $5.36 
Sell: $884.25 | Profit: -$16.71 
Sell: $895.76 | Profit: -$8.51 
Sell: $897.38 | Profit: -$12.45 
Sell: $892.47 | Profit: -$7.89 
Sell: $889.66 | Profit: -$7.08 
Sell: $875.40 | Profit: -$38.75 
Sell: $879.39 | Profit: -$54.37 
Sell: $879.82 | Profit: -$50.73 
Sell: $909.03 | Profit: -$23.84 
Sell: $908.59 | Profit: -$4.72 
Sell: $929.01 | Profit: -$9.86 
Sell: $922.93 | Profit: -$13.38 
Sell: $909.93 | Profit: -$24.60 
Sell: $927.57 | Profit: $6.82 
Buy: $847.48
Buy: $858.54
Buy: $864.36
Buy: $844.61
Sell: $855.70 | Profit: $8.22 
Sell: $860.32 | Profit: $1.78 
Sell: $848.20 | Profit: -$16.16 
Sell: $843.

Buy: $1172.53
Buy: $1171.42
Buy: $1174.28
Buy: $1165.36
Buy: $1181.41
Buy: $1180.59
Buy: $1172.92
Buy: $1176.12
Buy: $1181.39
Buy: $1184.07
Sell: $1191.14 | Profit: $3.44 
Sell: $1181.20 | Profit: $3.75 
Sell: $1181.21 | Profit: -$3.31 
Sell: $1187.76 | Profit: -$8.22 
Sell: $1173.79 | Profit: -$10.84 
Sell: $1162.05 | Profit: -$13.36 
Sell: $1142.62 | Profit: -$25.25 
Sell: $1145.98 | Profit: -$17.77 
Sell: $1152.78 | Profit: -$15.63 
Sell: $1137.50 | Profit: -$36.57 
Sell: $1159.95 | Profit: -$14.60 
Sell: $1152.12 | Profit: -$19.24 
Sell: $1162.10 | Profit: -$19.17 
Sell: $1151.83 | Profit: -$37.58 
Sell: $1156.38 | Profit: -$36.81 
Sell: $1143.22 | Profit: -$46.67 
Sell: $1156.85 | Profit: -$46.18 
Sell: $1162.16 | Profit: -$39.56 
Sell: $1161.17 | Profit: -$41.13 
Sell: $1175.65 | Profit: -$16.34 
Sell: $1172.63 | Profit: -$24.38 
Sell: $1171.35 | Profit: -$33.95 
Sell: $1178.84 | Profit: -$27.30 
Buy: $1166.22
Sell: $1171.11 | Profit: -$39.01 
Buy: $1159.36
Sell: $1154.05 | Profi

Sell: $1325.18 | Profit: $138.61 
Sell: $1318.03 | Profit: $127.93 
Sell: $1314.78 | Profit: $136.64 
Sell: $1326.37 | Profit: $130.61 
Sell: $1336.35 | Profit: $158.55 
Sell: $1336.59 | Profit: $157.00 
Sell: $1338.88 | Profit: $139.50 
Sell: $1335.85 | Profit: $139.31 
Sell: $1331.32 | Profit: $139.94 
Sell: $1334.11 | Profit: $155.21 
Sell: $1350.20 | Profit: $151.79 
Sell: $1353.22 | Profit: $146.21 
Sell: $1349.59 | Profit: $146.83 
Sell: $1350.66 | Profit: $135.90 
Sell: $1353.42 | Profit: $133.48 
Sell: $1349.95 | Profit: $129.81 
Sell: $1362.83 | Profit: $140.02 
Sell: $1365.62 | Profit: $147.03 
Sell: $1369.06 | Profit: $148.41 
Sell: $1364.05 | Profit: $133.09 
Sell: $1365.80 | Profit: $131.08 
Sell: $1366.96 | Profit: $133.20 
Sell: $1368.60 | Profit: $139.59 
Sell: $1377.02 | Profit: $145.81 
Sell: $1377.38 | Profit: $134.58 
Sell: $1382.22 | Profit: $133.95 
Sell: $1389.08 | Profit: $134.23 
Sell: $1377.34 | Profit: $116.11 
Sell: $1377.93 | Profit: $112.32 
Sell: $1377.94

Buy: $806.12
Buy: $813.88
Buy: $832.86
Buy: $815.94
Buy: $787.53
Buy: $797.87
Buy: $811.08
Buy: $834.38
Buy: $842.50
Buy: $835.48
Buy: $815.55
Buy: $825.16
Buy: $856.56
Buy: $858.73
Buy: $841.50
Buy: $852.06
Buy: $865.30
Buy: $869.60
Buy: $832.39
Buy: $850.08
Buy: $843.55
Buy: $851.92
Buy: $866.23
Buy: $857.51
Buy: $855.16
Buy: $873.64
Buy: $872.81
Buy: $877.52
Buy: $907.24
Buy: $903.80
Buy: $919.53
Buy: $907.39
Buy: $929.23
Buy: $909.24
Buy: $908.35
Buy: $883.92
Buy: $893.07
Buy: $882.88
Buy: $909.71
Buy: $908.13
Buy: $903.47
Buy: $888.33
Buy: $887.00
Buy: $910.33
Buy: $893.06
Buy: $906.83
Buy: $919.14
Buy: $942.87
Buy: $944.74
Buy: $931.76
Buy: $942.46
Buy: $940.09
Buy: $939.14
Buy: $942.43
Buy: $939.15
Buy: $944.89
Buy: $946.21
Buy: $923.72
Buy: $911.97
Buy: $910.71
Buy: $918.37
Buy: $921.23
Buy: $893.04
Buy: $895.10
Buy: $900.94
Buy: $920.26
Buy: $918.90
Buy: $927.23
Buy: $919.32
Buy: $923.33
Buy: $896.42
Buy: $898.72
Buy: $881.03
Buy: $879.56
Buy: $882.68
Buy: $879.13
Buy: $901.05

Buy: $1050.71
Buy: $1059.02
Sell: $1053.25 | Profit: $2.54 
Sell: $1051.81 | Profit: -$7.21 
Buy: $1130.52
Buy: $1132.05
Sell: $1139.83 | Profit: $9.31 
Sell: $1138.77 | Profit: $6.72 
Buy: $1091.49
Buy: $1088.68
Sell: $1089.19 | Profit: -$2.30 
Sell: $1093.56 | Profit: $4.88 
Buy: $1064.80
Buy: $1079.34
Sell: $1081.71 | Profit: $16.91 
Sell: $1095.17 | Profit: $15.83 
Buy: $1167.87
Sell: $1163.75 | Profit: -$4.12 
Buy: $1180.59
Buy: $1172.92
Sell: $1176.12 | Profit: -$4.47 
Sell: $1181.39 | Profit: $8.47 
Buy: $1223.29
Buy: $1226.50
Sell: $1227.92 | Profit: $4.63 
Sell: $1221.13 | Profit: -$5.37 
Buy: $1226.70
Buy: $1214.47
Sell: $1196.39 | Profit: -$30.31 
Sell: $1191.49 | Profit: -$22.98 
Buy: $1249.48
Buy: $1264.67
Sell: $1265.08 | Profit: $15.60 
Sell: $1262.09 | Profit: -$2.58 
Buy: $1289.12
Buy: $1285.33
Sell: $1307.28 | Profit: $18.16 
Sell: $1309.93 | Profit: $24.60 
Buy: $1366.96
Buy: $1368.60
Sell: $1377.02 | Profit: $10.06 
Sell: $1377.38 | Profit: $8.78 
Buy: $1387.17
Buy:

Buy: $1309.72
Sell: $1310.61 | Profit: $0.89 
Buy: $1316.28
Buy: $1319.66
Sell: $1321.18 | Profit: $4.90 
Sell: $1317.64 | Profit: -$2.02 
Buy: $1413.04
Buy: $1411.56
Sell: $1413.21 | Profit: $0.17 
Sell: $1425.49 | Profit: $13.93 
Buy: $1484.35
Buy: $1480.93
Sell: $1480.41 | Profit: -$3.94 
Sell: $1495.42 | Profit: $14.49 
Buy: $1433.06
Sell: $1467.67 | Profit: $34.61 
Buy: $1390.19
Sell: $1409.13 | Profit: $18.94 
Buy: $1385.35
Buy: $1390.84
Sell: $1398.26 | Profit: $12.91 
Sell: $1400.38 | Profit: $9.54 
Buy: $1252.31
Sell: $1273.70 | Profit: $21.39 
Buy: $888.67
Buy: $899.24
Sell: $873.59 | Profit: -$15.08 
Sell: $879.73 | Profit: -$19.51 
Buy: $778.94
Buy: $770.05
Sell: $743.33 | Profit: -$35.61 
Sell: $773.14 | Profit: $3.09 
Buy: $696.33
Sell: $712.87 | Profit: $16.54 
Buy: $951.13
Buy: $954.58
Sell: $954.07 | Profit: $2.94 
Sell: $976.29 | Profit: $21.71 
Buy: $1114.05
Buy: $1118.02
Sell: $1120.59 | Profit: $6.54 
Sell: $1126.48 | Profit: $8.46 
Buy: $1207.17
Buy: $1205.94
Sell

In [36]:
import keras
from keras.models import load_model
import sys

#if len(sys.argv) != 3:
#	print "Usage: python evaluate.py [stock] [model]"
#	exit()

#stock_name, model_name = sys.argv[1], sys.argv[2]
stock_name = "/home/tushar/Desktop/Python/Python3/q-trader/data/^GSPC_2011.csv"
model_name = "model_ep0"
model = load_model("/home/tushar/Desktop/Python/Python3/q-trader/models/" + model_name)
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

state = getState(data, 0, window_size + 1)
total_profit = 0
agent.inventory = []

for t in range(l):
    action = agent.act(state)
    # sit
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0

    if action == 1: # buy
        agent.inventory.append(data[t])
        print ("Buy: {}".format(formatPrice(data[t])))

    elif action == 2 and len(agent.inventory) > 0: # sell
        bought_price = agent.inventory.pop(0)
        reward = max(data[t] - bought_price, 0)
        total_profit += data[t] - bought_price
        print ("Sell: {} | Profit: {} ".format(formatPrice(data[t]),formatPrice(data[t] - bought_price)))

    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state

    if done:
        print ("--------------------------------")
        print ("{} Total Profit: {}".format(stock_name,formatPrice(total_profit)))
        print ("--------------------------------")

--------------------------------
/home/tushar/Desktop/Python/Python3/q-trader/data/^GSPC_2011.csv Total Profit: $0.00
--------------------------------
